In [17]:
%load_ext autoreload
%autoreload 2
%env PYTHONWARNINGS ignore

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: PYTHONWARNINGS=ignore


In [5]:
! pip uninstall simms -q -y
! pip install git+https://github.com/PangeAI/simms.git@main

  Cloning https://github.com/PangeAI/simms.git (to revision dev) to /tmp/pip-req-build-cmzzvn94
  Running command git clone --filter=blob:none --quiet https://github.com/PangeAI/simms.git /tmp/pip-req-build-cmzzvn94
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  branch 'dev' set up to track 'origin/dev'.
  Resolved https://github.com/PangeAI/simms.git to commit bcc415cd79d9891ffe4643f407b115bcfd943bdc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for simms: filename=simms-0.0.1-py3-none-any.whl size=24913 sha256=c4b23cd089228c6a5c285b10cbebbf7dee4e86ae4cc3d3d1104bc422036733bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ppooswx6/wheels/a4/f2/af/c0470cb2244b8ae55558b0a882bed091640dbaadcfdaf73b72
Successfully built simms


# Load data

In [118]:
from pathlib import Path
import numpy as np
from tqdm import tqdm
import torch
from numba import cuda
assert torch.cuda.is_available()
assert cuda.is_available()

from simms.utils import download
import pickle
from pathlib import Path
from matchms import calculate_scores

gnps = pickle.loads(
    Path(
        download('GNPS-LIBRARY-default-filter-nmax-2048.pickle')
    ).read_bytes()
)
batch_size = 512

In [136]:
%load_ext autoreload
%autoreload 2
from simms.similarity import CudaCosineGreedy
from matchms import calculate_scores

kernel = CudaCosineGreedy(
    tolerance=.1,
    shift=0,
    mz_power=0,
    intensity_power=1,
    match_limit=2048,
    batch_size=batch_size,
    sparse_threshold=.75
)

references=gnps[:batch_size]
queries=gnps[:batch_size]

sparse_results = calculate_scores(
    references=references,
    queries=queries,
    similarity_function=kernel,
    array_type='sparse',
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[(Spectrum(precursor m/z=940.25, 335 fragments between 278.0 and 940.9),
  (1., 335, False))]

In [122]:
from matchms.similarity import CosineGreedy

expected_results = calculate_scores(
    references=references,
    queries=queries,
    similarity_function=CosineGreedy(),
)

In [125]:
a = expected_results.to_array('CosineGreedy_score')

In [130]:
b = sparse_results.to_array('CudaCosineGreedy_sparse_score')

In [132]:
np.isclose(a, b).mean()

0.9999923706054688

In [ ]:
sparse_results.

In [135]:
print(sparse_results.score_names)
print(sparse_results.scores_by_reference(
    reference=references[3],
    name='score'
))

('CudaCosineGreedy_sparse_score', 'CudaCosineGreedy_sparse_matches', 'CudaCosineGreedy_sparse_overflow')
[(Spectrum(precursor m/z=545.15, 302 fragments between 160.5 and 1955.4), (0.9240435, 18, False))]


In [53]:
query = queries[15]  # just an example
best_matches = sparse_results.scores_by_query(query, "CosineGreedy_score", sort=True)

ValueError: no field of name CosineGreedy_score

In [43]:
sparse_results

<1024x1024x5 stacked sparse array containing scores for ('CudaCosineGreedy_q_idx', 'CudaCosineGreedy_r_idx', 'CudaCosineGreedy_score', 'CudaCosineGreedy_matches', 'CudaCosineGreedy_overflow') with 198 stored elements in COOrdinate format>

We will run a pairwise cosine similarity on the entirety of the GNPS dataset (around 500_000 spectra).

Parsing these many spectra takes a while, so I already have a pickled version of the same dataset ready to go in `ALL_GNPS.pickle`.

Alternatively, you can use `ALL_GNPS.mgf` and wait for the parsing to finish.

In [8]:
from simms.utils import download
from pathlib import Path
from joblib import Parallel, delayed
from matchms.filtering import default_filters, normalize_intensities, reduce_to_number_of_peaks
from matchms.importing import load_from_mgf
import pickle

spectra_file = download('ALL_GNPS.mgf')
def parse_spectrum(spectrum):
    ## Uncomment if you want default filters enabled - add more if you need them.
    # spectrum = default_filters(spectrum)
    spectrum = reduce_to_number_of_peaks(spectrum, n_max=max_peaks)
    # spectrum = normalize_intensities(spectrum)
    return spectrum

limit = 200_000
spectrums = tuple(s for _, s in zip(range(limit), load_from_mgf(spectra_file)))
spectrums = Parallel(-1)(delayed(parse_spectrum)(spec) for spec in tqdm(spectrums))
spectrums = [spe for spe in spectrums if spe is not None]
## Download and read prepared pickle
# spectra_file = download('ALL_GNPS.pickle')
# spectrums = tuple(pickle.load(open(spectra_file, 'rb')))

 14%|█▍        | 2028/14244 [00:03<00:06, 1911.97it/s]

2024-02-29 16:29:22,483:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-02-29 16:29:22,490:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-02-29 16:29:22,575:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode


 25%|██▌       | 3564/14244 [00:03<00:05, 1860.61it/s]

2024-02-29 16:29:23,333:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,343:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,349:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,359:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,370:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,377:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,387:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,399:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29 16:29:23,404:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+Fe-2H]
2024-02-29

 28%|██▊       | 3948/14244 [00:04<00:05, 1869.48it/s]

...

 84%|████████▍ | 12012/14244 [00:08<00:01, 1838.91it/s]

2024-02-29 16:29:28,101:WARNING:matchms:derive_adduct_from_name:Two potential adducts were found in the compound name that are both valid adducts. The first adduct is used. The adducts found are: ['[M+Na]+', '[M+H]+']
2024-02-29 16:29:28,107:WARNING:matchms:derive_adduct_from_name:Two potential adducts were found in the compound name that are both valid adducts. The first adduct is used. The adducts found are: ['[M+Na]+', '[M+H]+']
2024-02-29 16:29:28,171:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-02-29 16:29:28,186:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode
2024-02-29 16:29:28,223:WARNING:matchms:interpret_unknown_adduct:Charge was found 0 times in adduct [M+2H+]


 95%|█████████▌| 13548/14244 [00:09<00:00, 1837.41it/s]

2024-02-29 16:29:29,002:WARNING:matchms:correct_charge:Changed sign of given charge: -1 to match positive ionmode


100%|██████████| 14244/14244 [00:09<00:00, 1463.88it/s]


In [50]:
# Pairwise similarity between all
references, queries = spectrums[:100_000], spectrums[100_000:]

print(f"We have {len(references) + len(queries):.3e} spectra")
print(f"Pairwise comparisons have {len(references)*len(queries):.3e} pairs in total")

We have 2.000e+05 spectra
Pairwise comparisons have 1.000e+10 pairs in total


In [51]:
from simms.utils import spectra_peaks_to_tensor
from itertools import product
dtype = np.float32
padding = None

batches_r = []
for bstart, bend in tqdm(
    argbatch(references, batch_size), desc="Batch all references",
    total=len(references)//batch_size
):
    rbatch = references[bstart:bend]
    rspec, rlen = spectra_peaks_to_tensor(rbatch, dtype=dtype)
    batches_r.append([rspec, rlen, bstart, bend])

batches_q = []
for bstart, bend in tqdm(
    argbatch(queries, batch_size), desc="Batch all queries",
    total=len(queries)//batch_size
):
    qbatch = queries[bstart:bend]
    qspec, qlen = spectra_peaks_to_tensor(qbatch, dtype=dtype)
    batches_q.append([qspec, qlen, bstart, bend])
    
batched_inputs = tuple(product(batches_r, batches_q))

Batch all references: 13it [00:01,  6.88it/s]                        
Batch all queries: 13it [00:02,  5.27it/s]                        


In [52]:
device = torch.device('cuda')
host = torch.device('cpu')

! rm -rf data/output
! mkdir -p data/output

with torch.no_grad():
    for batch_i in tqdm(range(len(batched_inputs)), smoothing=1):
        (rspec, rlen, rstart, rend), (qspec, qlen, qstart, qend) = batched_inputs[
            batch_i
        ]
        
        lens = torch.zeros(2, batch_size, dtype=torch.int32)
        lens[0, :len(rlen)] = torch.from_numpy(rlen)
        lens[1, :len(qlen)] = torch.from_numpy(qlen)
        
        lens = lens.to(device)
        
        rspec = torch.from_numpy(rspec).to(device)
        qspec = torch.from_numpy(qspec).to(device)
    
        rspec = cuda.as_cuda_array(rspec)
        qspec = cuda.as_cuda_array(qspec)
        lens = cuda.as_cuda_array(lens)
            
        out = torch.empty(3, batch_size, batch_size, dtype=torch.float32, device=device)
        out = cuda.as_cuda_array(out)
        
        kernel(rspec, qspec, lens, out)
        
        out = torch.as_tensor(out, device=device)
        mask = out[0] >= threshold
        
        row, col = torch.nonzero(mask, as_tuple=True)
        rabs = rstart + row
        qabs = qstart + col
        score, matches, overflow = out[:, mask].to(host)
        
        np.savez_compressed(
            f'data/output/{rstart}-{rend}-{qstart}-{qend}.npz', 
            rabs=rabs.int().to(host), 
            qabs=qabs.int().to(host), 
            score=score.float(),
            matches=matches.int(),
            overflow=overflow.bool()
        )

  1%|          | 1/169 [00:01<02:54,  1.04s/it]

tensor(0.0006, device='cuda:0') tensor(0.0033)


  1%|          | 2/169 [00:01<01:52,  1.48it/s]

tensor(0.0006, device='cuda:0') tensor(0.0038)


  2%|▏         | 3/169 [00:02<01:56,  1.43it/s]

...

tensor(0.0001, device='cuda:0') tensor(0.0007)
tensor(0.0001, device='cuda:0') tensor(0.0007)


 98%|█████████▊| 166/169 [02:56<00:00,  6.97it/s]

tensor(0.0001, device='cuda:0') tensor(0.0006)
tensor(0.0002, device='cuda:0') tensor(0.0008)


100%|██████████| 169/169 [02:56<00:00,  1.04s/it]

tensor(0.0002, device='cuda:0') tensor(0.0007)
tensor(0.0002, device='cuda:0') tensor(0.0008)
tensor(3.8624e-05, device='cuda:0') tensor(0.0002)


In [53]:
! du -hs data/output/

159M	data/output/


In [54]:
from pathlib import Path

! du -hs data/output/

total_size = sum(f.stat().st_size for f in Path('data/output').glob('**/*') if f.is_file())
print(f'Total file size {total_size/1e9:.3f} GB')

159M	data/output/
Total file size 0.166 GB


In [55]:
qabs = []
rabs = []
score = []
matches = []
overflow = []
for file in tqdm(Path('data/output').glob('*.npz')):
    bunch = np.load(file)
    qabs += [bunch['qabs']]
    rabs += [bunch['rabs']]
    score += [bunch['score']]
    matches += [bunch['matches']]
    overflow += [bunch['overflow']]

169it [00:01, 93.24it/s] 


In [56]:
qabs = np.concatenate(qabs)
rabs = np.concatenate(rabs)
score = np.concatenate(score)
matches = np.concatenate(matches)
overflow = np.concatenate(overflow)

In [57]:
# Suppose we want to query these absolute query IDs, and sort their results
query = np.array([1, 42, 121, 99_999])

In [58]:
import pandas as pd
from IPython.display import display
for q in query:
    idx = qabs == q
    res = np.stack([rabs[idx], score[idx], matches[idx], overflow[idx]],axis=1)
    res = pd.DataFrame(res, columns='ReferenceID Score Matches Overflow'.split())
    print(f"Similarity for chemical with QueryID={q}")
    display(res)

Similarity for chemical with QueryID=1


,ReferenceID,Score,Matches,Overflow
0,1071.0,0.774170,12.0,0.0
1,1671.0,0.793073,3.0,0.0
2,4898.0,0.770358,1.0,0.0
3,4900.0,0.788730,1.0,0.0
4,5200.0,0.784787,2.0,0.0
...,...,...,...,...
11727,98233.0,0.756576,1.0,0.0
11728,98238.0,0.786356,1.0,0.0
11729,98567.0,0.754935,1.0,0.0
11730,98569.0,0.790264,1.0,0.0


Similarity for chemical with QueryID=42


,ReferenceID,Score,Matches,Overflow
0,1524.0,0.975595,1.0,0.0
1,2112.0,0.977732,1.0,0.0
2,3188.0,0.954756,2.0,0.0
3,3531.0,0.829793,22.0,0.0
4,3534.0,0.950818,14.0,0.0
...,...,...,...,...
120,92470.0,0.951438,4.0,0.0
121,92518.0,0.979738,2.0,0.0
122,92706.0,0.916761,1.0,0.0
123,96856.0,0.973249,1.0,0.0


Similarity for chemical with QueryID=121


,ReferenceID,Score,Matches,Overflow
0,2306.0,0.859755,17.0,0.0
1,35221.0,0.796745,2.0,0.0
2,38910.0,0.834682,1.0,0.0
3,60718.0,0.895202,5.0,0.0
4,74810.0,0.943186,3.0,0.0
5,93089.0,0.975652,14.0,0.0


Similarity for chemical with QueryID=99999


,ReferenceID,Score,Matches,Overflow
0,5510.0,0.772368,3.0,0.0
1,11319.0,0.776063,11.0,0.0
2,14269.0,0.756477,2.0,0.0
3,14856.0,0.781213,2.0,0.0
4,14857.0,0.776356,2.0,0.0
5,15047.0,0.781366,8.0,0.0
6,15048.0,0.775313,6.0,0.0
7,16079.0,0.774708,2.0,0.0
8,20837.0,0.768895,7.0,0.0
9,23569.0,0.774484,20.0,0.0
